## Discarded due to lack of computation power

In [1]:
import pandas as pd
import numpy as np
import csv
from sklearn.model_selection import train_test_split
import gc
import pickle
import joblib
import os

In [2]:
train = pd.read_csv('dataset/train.csv', escapechar = "\\", quoting = csv.QUOTE_NONE, dtype = {'BROWSE_NODE_ID' : np.uint16})

In [3]:
train.head()

,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","Pete the Cat is the coolest, most popular cat ...","[Pete the Cat Bedtime Blues plush doll,Based o...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",The New Yorker Handsome Cello Wrapped Hard Mag...,[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,NaN,Skyhorse Publishing,imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),NaN,"[Nutrilite Kids,Chewable Iron Tablets,Quantity...",Amway,3
4,Teacher Planner Company A4 6 Lesson Academic T...,NaN,NaN,NaN,4


In [4]:
train.shape

(2903024, 5)

In [5]:
# Drop rows with more than 2 null values
train_thresh = train.dropna(thresh=3)
train_thresh.head()

,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","Pete the Cat is the coolest, most popular cat ...","[Pete the Cat Bedtime Blues plush doll,Based o...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",The New Yorker Handsome Cello Wrapped Hard Mag...,[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,NaN,Skyhorse Publishing,imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),NaN,"[Nutrilite Kids,Chewable Iron Tablets,Quantity...",Amway,3
5,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,"[Color: Blue,Sleeve: Full Sleeve,Material: Cot...",Bhavya Enterprise,5


In [6]:
del train
gc.collect()

0

In [7]:
train_thresh.shape

(2855971, 5)

In [8]:
train_thresh.dtypes

TITLE             object
DESCRIPTION       object
BULLET_POINTS     object
BRAND             object
BROWSE_NODE_ID    uint16
dtype: object

In [9]:
# Check number of null values in each column
train_thresh.isnull().sum(axis = 0)

TITLE                 60
DESCRIPTION       676612
BULLET_POINTS     119210
BRAND               9685
BROWSE_NODE_ID         0
dtype: int64

In [10]:
# Count number of classes / product browse nodes
len(train_thresh['BROWSE_NODE_ID'].unique())

9334

In [11]:
# Replace NaN values with space
train_replaced_NaN = train_thresh.replace(np.nan, '', regex=True)

In [12]:
del train_thresh
gc.collect()

0

In [13]:
train_replaced_NaN.head()

,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","Pete the Cat is the coolest, most popular cat ...","[Pete the Cat Bedtime Blues plush doll,Based o...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",The New Yorker Handsome Cello Wrapped Hard Mag...,[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,,Skyhorse Publishing,imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),,"[Nutrilite Kids,Chewable Iron Tablets,Quantity...",Amway,3
5,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,"[Color: Blue,Sleeve: Full Sleeve,Material: Cot...",Bhavya Enterprise,5


In [14]:
# Split dataset into train and validation
X = train_replaced_NaN[['TITLE', 'DESCRIPTION', 'BULLET_POINTS', 'BRAND']]
y = train_replaced_NaN["BROWSE_NODE_ID"]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size = 0.8, test_size = 0.2, random_state = 0)

In [15]:
""" del train_replaced_NaN
gc.collect() """
del X
del y
gc.collect()

0

In [16]:
# Corpus generation function
def generateCorpus(df):
    np_arr = df.to_numpy()
    corpus = []
    for product_desc in np_arr:
        str = ""
        for desc in product_desc:
            str+=desc
            str+= ', '
        corpus.append(str + '\n')
    return corpus

In [17]:
train_corpus = generateCorpus(X_train)

In [18]:
del X_train
gc.collect()

0

In [ ]:
train_corpus[:3]

In [20]:
""" import os
txt_file = open("./corpus/train_corpus.txt","w", encoding = "utf8")
for desc in train_corpus:
    txt_file.write (desc + "\n")
txt_file.close() """

' import os\ntxt_file = open("./corpus/train_corpus.txt","w", encoding = "utf8")\nfor desc in train_corpus:\n    txt_file.write (desc + "\n")\ntxt_file.close() '

In [21]:
len(train_corpus)

2284776

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

In [23]:
model=make_pipeline(TfidfVectorizer(),MultinomialNB())
model.fit(train_corpus,y_train)
joblib.dump(model, './models/naiveBayes.pkl')
print("hello saved model")

In [ ]:
""" labels=model.predict(X_valid) """

Error: Session cannot generate requests

In [ ]:
""" from sklearn.metrics import mean_absolute_error """

In [ ]:
""" mae = mean_absolute_error(labels, y_valid)
print(mae) """